# TD2 : Réseaux de neurones convolutionnels — Notebook optimisé (Colab)

**Objectif :** ce notebook suit *exactement* la structure et l'intitulé des sous‑questions du PDF, avec **une sous‑question par cellule de code**, prêt à être exécuté sur **Google Colab**.

In [ ]:
# --- Préparation de l'environnement ---
# Notebook conçu pour être exécutable "Run all" dans Google Colab.
# En local, assurez-vous d'avoir TensorFlow et TensorFlow Datasets installés.
#
# (Optionnel - Colab) Si besoin, décommentez :
# !pip -q install -U tensorflow tensorflow_datasets matplotlib

import os
import math
import random

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers as L

# Reproductibilité (dans la limite des ops GPU non-déterministes)
SEED = 42
keras.utils.set_random_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

AUTOTUNE = tf.data.AUTOTUNE
print("TF:", tf.__version__)
print("TFDS:", tfds.__version__)
print("Devices:", tf.config.list_logical_devices())


# Exercice 1

### 1. Charger le dataset CIFAR-10

In [ ]:
# Exercice 1 — 1. Charger le dataset CIFAR-10 (train/val/test)
# Bonne pratique : ne pas utiliser le test comme validation (évite la fuite de données).
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train[:90%]", "train[90%:]", "test"],
    with_info=True,
    as_supervised=True,
)

num_classes = ds_info.features["label"].num_classes
input_shape = ds_info.features["image"].shape
print(
    "Classes:",
    num_classes,
    "| Input shape:",
    input_shape,
    "| Train:",
    tf.data.experimental.cardinality(ds_train).numpy(),
    "| Val:",
    tf.data.experimental.cardinality(ds_val).numpy(),
    "| Test:",
    tf.data.experimental.cardinality(ds_test).numpy(),
)


### 2. Préparer le dataset pour l’apprentissage : normaliser les images, encoder les sorties pour un réseau de neurones

In [ ]:
# Exercice 1 — 2. Préparation (normalisation + encodage one-hot)
def preprocess_ex1(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, depth=num_classes)
    return image, label


BATCH = 128

ds_train_prep = (
    ds_train.map(preprocess_ex1, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(10_000, seed=SEED, reshuffle_each_iteration=True)
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)

ds_val_prep = (
    ds_val.map(preprocess_ex1, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)

ds_test_prep = (
    ds_test.map(preprocess_ex1, num_parallel_calls=AUTOTUNE)
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)


### 3. Implémenter et tester l’architecture suivante :

In [ ]:
# Exercice 1 — 3. CNN de base selon la spécification
def build_cnn_ex1(input_shape=(32, 32, 3), num_classes=10):
    model = keras.Sequential(
        [
            L.Input(shape=input_shape),
            L.Conv2D(32, 3, padding="same", activation="relu"),
            L.MaxPooling2D(2),
            L.Conv2D(64, 3, padding="same", activation="relu"),
            L.MaxPooling2D(2),
            L.Flatten(),
            L.Dense(256, activation="relu"),
            L.Dense(num_classes, activation="softmax"),
        ]
    )
    return model


model_ex1 = build_cnn_ex1(input_shape=input_shape, num_classes=num_classes)
model_ex1.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model_ex1.summary()
history_ex1 = model_ex1.fit(
    ds_train_prep, validation_data=ds_val_prep, epochs=10, verbose=2
)


# Évaluation finale sur le test
test_loss, test_acc = model_ex1.evaluate(ds_test_prep, verbose=0)
print(f"Test — loss: {test_loss:.4f} | acc: {test_acc:.4f}")


### 4. Comparer les résultats de l’apprentissage avec les MLP développés dans le TD précédent, en performance et en nombre de paramètres. Evaluer la capacité de ce réseau à généraliser de cette architecture.

In [ ]:
# Exercice 1 — 4. Comparaison (indicative)
mlp_params = None  # Remplacez par votre valeur du TD précédent
mlp_val_acc = None  # Remplacez par votre valeur du TD précédent

cnn_params = model_ex1.count_params()
cnn_val_acc = history_ex1.history.get("val_accuracy", [None])[-1]
print(f"[CNN] params={cnn_params:,} | val_acc={cnn_val_acc}")
print(f"[MLP] params={mlp_params} | val_acc={mlp_val_acc}")
print(
    "Observation: le CNN exploite la structure spatiale et généralise souvent mieux qu’un MLP de taille comparable."
)


### 5. Modifier l’architecture précédente pour améliorer les résultats obtenus.

In [ ]:
# Exercice 1 — 5. CNN amélioré
data_aug = keras.Sequential(
    [
        L.RandomFlip("horizontal"),
        L.RandomRotation(0.05),
    ]
)


def build_cnn_ex1_improved(input_shape=(32, 32, 3), num_classes=10):
    inputs = L.Input(shape=input_shape)
    x = data_aug(inputs)
    x = L.Conv2D(32, 3, padding="same", activation=None)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.MaxPooling2D(2)(x)
    x = L.Conv2D(64, 3, padding="same", activation=None)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.MaxPooling2D(2)(x)
    x = L.Conv2D(128, 3, padding="same", activation=None)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.3)(x)
    outputs = L.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


model_ex1_imp = build_cnn_ex1_improved(input_shape=input_shape, num_classes=num_classes)
model_ex1_imp.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model_ex1_imp.summary()
history_ex1_imp = model_ex1_imp.fit(
    ds_train_prep, validation_data=ds_val_prep, epochs=10, verbose=2
)

# Évaluation finale sur le test
test_loss, test_acc = model_ex1_imp.evaluate(ds_test_prep, verbose=0)
print(f"Test (improved) — loss: {test_loss:.4f} | acc: {test_acc:.4f}")


# Exercice 2

### 1. Télécharger le dataset ‘stanford_dogs’ directement depuis votre notebook :

In [ ]:
# Exercice 2 — 1. Télécharger le dataset ‘stanford_dogs’
(ds_train_dogs, ds_test_dogs), ds_info_dogs = tfds.load(
    "stanford_dogs", split=["train", "test"], with_info=True, as_supervised=True
)
print(ds_info_dogs)

### 2. Récupérez le nombre de classes à partir des informations. Combien y at’il de classes ?

In [ ]:
# Exercice 2 — 2. Nombre de classes
num_classes_dogs = ds_info_dogs.features["label"].num_classes
print("Nombre de classes (stanford_dogs):", num_classes_dogs)

### 3. Visualisez les premières images du dataset à l’aide de la méthode show_examples

In [ ]:
# Exercice 2 — 3. Visualisation d'exemples
tfds.visualization.show_examples(ds_train_dogs, ds_info_dogs)

### 4. Récupérez la première image.

In [ ]:
# Exercice 2 — 4. Récupérer la première image
sample_image, sample_label = next(iter(ds_train_dogs.take(1)))
print(
    "Image dtype/shape:",
    sample_image.dtype,
    sample_image.shape,
    "| Label:",
    sample_label.numpy(),
)

### 5. Convertissez cette image en grayscale (nécessite d’installer tensorflow_io) avec rgb_to_grayscale.

In [ ]:
# Exercice 2 — 5. Conversion en niveaux de gris (TensorFlow standard)
gray = tf.image.rgb_to_grayscale(tf.cast(sample_image, tf.float32))
print("Grayscale shape:", gray.shape)
plt.imshow(tf.squeeze(gray).numpy(), cmap="gray")
plt.axis("off")
plt.show()


### 6. ... Ajoutez une dimension avec expand_dims, puis vérifiez le résultat.

In [ ]:
# Exercice 2 — 6. Ajouter la dimension batch
print("Avant:", gray.shape)
gray_batched = tf.expand_dims(gray, axis=0)
print("Après :", gray_batched.shape)

### 7. Définissez un noyau de convolution ... 3x3 en float32 (valeurs manuelles).

In [ ]:
# Exercice 2 — 7. Noyau 3x3 manuel
import numpy as np

kernel_2d = np.array(
    [
        [-1.0, 0.0, 1.0],
        [-2.0, 0.0, 2.0],
        [-1.0, 0.0, 1.0],
    ],
    dtype=np.float32,
)
print(kernel_2d)

### 8. ... appliquer le noyau avec tf.nn.conv2d (stride=1, padding='SAME'), puis visualiser en grayscale.

In [ ]:
# Exercice 2 — 8. Application du filtre via conv2d
kernel_4d = tf.reshape(kernel_2d, [3, 3, 1, 1])
filtered = tf.nn.conv2d(gray_batched, kernel_4d, strides=[1, 1, 1, 1], padding="SAME")
filtered_img = tf.squeeze(filtered, axis=0)
plt.imshow(tf.squeeze(filtered_img).numpy(), cmap="gray")
plt.axis("off")
plt.show()

### 9. ... implémente un filtre de Sobel.

In [ ]:
# Exercice 2 — 9. Filtre de Sobel
sobel_x = tf.reshape(
    tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32), [3, 3, 1, 1]
)
sobel_y = tf.reshape(
    tf.constant([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=tf.float32), [3, 3, 1, 1]
)
gx = tf.nn.conv2d(gray_batched, sobel_x, strides=1, padding="SAME")
gy = tf.nn.conv2d(gray_batched, sobel_y, strides=1, padding="SAME")
mag = tf.sqrt(tf.square(gx) + tf.square(gy) + 1e-6)
plt.figure(figsize=(10, 3))
plt.subplot(1, 3, 1)
plt.title("Gx")
plt.imshow(tf.squeeze(gx), cmap="gray")
plt.axis("off")
plt.subplot(1, 3, 2)
plt.title("Gy")
plt.imshow(tf.squeeze(gy), cmap="gray")
plt.axis("off")
plt.subplot(1, 3, 3)
plt.title("Magnitude")
plt.imshow(tf.squeeze(mag), cmap="gray")
plt.axis("off")
plt.show()

# Exercice 3

### 1. Télécharger le dataset ‘stanford_dogs‘ depuis votre notebook avec les mêmes options que dans l’exercice précédent.

In [ ]:
# Exercice 3 — 1. Télécharger stanford_dogs
(ds_train_dogs2, ds_test_dogs2), ds_info_dogs2 = tfds.load(
    "stanford_dogs", split=["train", "test"], with_info=True, as_supervised=True
)
n_classes = ds_info_dogs2.features["label"].num_classes
print("Classes:", n_classes)

### 2. ... convertir les labels avec un encodage ‘one_hot’. Pour réaliser cela, utiliser map et tf.one_hot.

In [ ]:
# Exercice 3 — 2. One-hot des labels
def one_hot_map(image, label):
    return image, tf.one_hot(label, depth=n_classes)


dogs_train_oh = ds_train_dogs2.map(one_hot_map, num_parallel_calls=AUTOTUNE)
dogs_test_oh = ds_test_dogs2.map(one_hot_map, num_parallel_calls=AUTOTUNE)
for img, lab in dogs_train_oh.take(1):
    print("Image:", img.shape, "| Label one-hot shape:", lab.shape)


### 3. ... images 128x128 + normaliser [0,1] via Sequential(Resizing, Rescaling). Tester d’abord sur une image.

In [ ]:
# Exercice 3 — 3. Préprocessing (test sur une image)
preproc = keras.Sequential(
    [
        L.Resizing(128, 128),
        L.Rescaling(1.0 / 255.0),
    ]
)
img0, lab0 = next(iter(dogs_train_oh.take(1)))
img0_p = preproc(tf.expand_dims(img0, 0))
print(
    "Avant:",
    img0.shape,
    "Après:",
    img0_p.shape,
    "min/max:",
    tf.reduce_min(img0_p).numpy(),
    tf.reduce_max(img0_p).numpy(),
)

### 4. Appliquez les transformations à l’ensemble du dataset à l’aide de la fonction map.

In [ ]:
# Exercice 3 — 4. Appliquer au dataset complet
def pp_map(image, label_oh):
    image = preproc(image)
    return image, label_oh


BATCH_DOGS = 64
dogs_train_pp = (
    dogs_train_oh.map(pp_map, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(5000)
    .batch(BATCH_DOGS)
    .prefetch(AUTOTUNE)
)
dogs_test_pp = (
    dogs_test_oh.map(pp_map, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_DOGS)
    .prefetch(AUTOTUNE)
)
for img, lab in dogs_train_pp.take(1):
    print("Batch:", img.shape, lab.shape)

### 5. ... augmentation de données via couches de preprocessing (expérimentez sur une image).

In [ ]:
# Exercice 3 — 5. Augmentations sur une image
aug = keras.Sequential(
    [
        L.RandomFlip("horizontal"),
        L.RandomRotation(0.1),
        L.RandomContrast(0.1),
        L.RandomZoom(0.1),
    ]
)
augmented = aug(img0_p, training=True)
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title("Original (pp)")
plt.imshow(tf.squeeze(img0_p))
plt.axis("off")
plt.subplot(1, 2, 2)
plt.title("Augmentée")
plt.imshow(tf.squeeze(augmented))
plt.axis("off")
plt.show()

# Exercice 4

### 1. Téléchargez et préparez le dataset ‘beans’ pour l’entrainement.

In [ ]:
# Exercice 4 — 1. Télécharger et préparer 'beans'
(ds_train_beans, ds_val_beans, ds_test_beans), ds_info_beans = tfds.load(
    "beans", split=["train", "validation", "test"], with_info=True, as_supervised=True
)
n_classes_beans = ds_info_beans.features["label"].num_classes
input_shape_beans = ds_info_beans.features["image"].shape


def pp_beans(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, tf.one_hot(label, depth=n_classes_beans)


BATCH_BEANS = 64
train_beans = (
    ds_train_beans.map(pp_beans, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(5000)
    .batch(BATCH_BEANS)
    .prefetch(AUTOTUNE)
)
val_beans = (
    ds_val_beans.map(pp_beans, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_BEANS)
    .prefetch(AUTOTUNE)
)
test_beans = (
    ds_test_beans.map(pp_beans, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_BEANS)
    .prefetch(AUTOTUNE)
)
print("Beans | classes:", n_classes_beans, "| input:", input_shape_beans)


### 2. Implémentez une fonction qui retourne un réseau de neurones convolutionnel (style VGG, 3 blocs).

In [ ]:
# Exercice 4 — 2. VGG-like (3 blocs)
def build_vgg_beans(input_shape=(None, None, 3), num_classes=3):
    return keras.Sequential(
        [
            L.Input(shape=input_shape),
            # Bloc 1
            L.Conv2D(32, 3, padding="same", activation="relu"),
            L.Conv2D(32, 3, padding="same", activation="relu"),
            L.MaxPooling2D(2),
            # Bloc 2
            L.Conv2D(64, 3, padding="same", activation="relu"),
            L.Conv2D(64, 3, padding="same", activation="relu"),
            L.MaxPooling2D(2),
            # Bloc 3
            L.Conv2D(128, 3, padding="same", activation="relu"),
            L.Conv2D(128, 3, padding="same", activation="relu"),
            L.MaxPooling2D(2),
            L.GlobalAveragePooling2D(),
            L.Dense(128, activation="relu"),
            L.Dense(num_classes, activation="softmax"),
        ]
    )


model_beans = build_vgg_beans(
    input_shape=input_shape_beans, num_classes=n_classes_beans
)
model_beans.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model_beans.summary()


### 3. Compilez votre réseau ... La première couche convolutionnelle doit contenir 896 paramètres et la seconde 9248. Pourquoi ces nombres ? Comment calculer le nombre de paramètres d’une couche convolutionnelle ?

In [ ]:
# Exercice 4 — 3. Vérifier les paramètres
# Formule: params = (Kh * Kw * Cin + bias) * Cout ; bias=1 si bias présent.
# Conv1: (3*3*3 + 1) * 32 = 896 ; Conv2: (3*3*32 + 1) * 32 = 9248
conv1_weights = model_beans.layers[1].count_params()
conv2_weights = model_beans.layers[2].count_params()
print("Conv1 params (attendu 896):", conv1_weights)
print("Conv2 params (attendu 9248):", conv2_weights)


### 4. Entrainez le sur le dataset avec fit pendant 50 epochs ... récupérer history.

In [ ]:
# Exercice 4 — 4. Entraînement (50 epochs) avec validation 10%
EPOCHS = 50
history_beans = model_beans.fit(
    train_beans, validation_data=val_beans, epochs=EPOCHS, verbose=2
)


### 5. ... tracer des courbes d’évolution de la loss et de l’accuracy, sur les ensembles d’entrainement et de validation.

In [ ]:
# Exercice 4 — 5. Tracer loss/accuracy
epochs = range(1, len(history_beans.history["loss"]) + 1)
plt.figure()
plt.plot(epochs, history_beans.history["loss"], label="loss (train)")
plt.plot(epochs, history_beans.history["val_loss"], label="loss (val)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Beans — Loss")
plt.show()
plt.figure()
plt.plot(epochs, history_beans.history["accuracy"], label="acc (train)")
plt.plot(epochs, history_beans.history["val_accuracy"], label="acc (val)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Beans — Accuracy")
plt.show()


### 6. Pour finir, testez le réseau sur des données non observées lors de l’entrainement. Le résultat est-il satisfaisant ?

In [ ]:
# Exercice 4 — 6. Évaluation sur le test (non vu)
test_loss, test_acc = model_beans.evaluate(test_beans, verbose=0)
print(f"Test — loss: {test_loss:.4f} | acc: {test_acc:.4f}")


# Exercice 5

### 1. Sans changer l’architecture globale que vous aviez choisi, améliorez les performances de votre réseau ...

In [ ]:
# Exercice 5 — 1. Amélioration sans changer l'architecture globale
data_aug_beans = keras.Sequential(
    [
        L.RandomFlip("horizontal"),
        L.RandomRotation(0.1),
    ]
)


def build_vgg_beans_improved(input_shape=(None, None, 3), num_classes=3):
    inputs = L.Input(shape=input_shape)
    x = data_aug_beans(inputs)
    # Bloc 1
    x = L.Conv2D(32, 3, padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.Conv2D(32, 3, padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.MaxPooling2D(2)(x)
    x = L.Dropout(0.2)(x)
    # Bloc 2
    x = L.Conv2D(64, 3, padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.Conv2D(64, 3, padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.MaxPooling2D(2)(x)
    x = L.Dropout(0.3)(x)
    # Bloc 3
    x = L.Conv2D(128, 3, padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.Conv2D(128, 3, padding="same", use_bias=False)(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.4)(x)
    outputs = L.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


model_beans_imp = build_vgg_beans_improved(
    input_shape=input_shape_beans, num_classes=n_classes_beans
)
model_beans_imp.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model_beans_imp.summary()


### 2. Entrainez à nouveau votre réseau et réglez les differents paramètres pour obtenir un apprentissage satisfaisant, avec des courbes d’entrainement et de validation proches l’une de l’autre.

In [ ]:
# Exercice 5 — 2. Entraînement + visualisation
EPOCHS_IMP = 30
callbacks = [
    keras.callbacks.EarlyStopping(
        patience=6, restore_best_weights=True, monitor="val_accuracy"
    )
]
history_beans_imp = model_beans_imp.fit(
    train_beans,
    validation_data=val_beans,
    epochs=EPOCHS_IMP,
    callbacks=callbacks,
    verbose=2,
)
epochs_imp = range(1, len(history_beans_imp.history["loss"]) + 1)
plt.figure()
plt.plot(epochs_imp, history_beans_imp.history["loss"], label="loss (train)")
plt.plot(epochs_imp, history_beans_imp.history["val_loss"], label="loss (val)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Beans (amélioré) — Loss")
plt.show()
plt.figure()
plt.plot(epochs_imp, history_beans_imp.history["accuracy"], label="acc (train)")
plt.plot(epochs_imp, history_beans_imp.history["val_accuracy"], label="acc (val)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Beans (amélioré) — Accuracy")
plt.show()
test_loss_imp, test_acc_imp = model_beans_imp.evaluate(test_beans, verbose=0)
print(f"[Amélioré] Test — loss: {test_loss_imp:.4f} | acc: {test_acc_imp:.4f}")
